In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from darts import TimeSeries
from darts.models import TSMixerModel
from darts.metrics import mse, rmse, r2_score, mae, smape
from darts.dataprocessing.transformers import Scaler
import torch
from optuna.integration import PyTorchLightningPruningCallback
from pytorch_lightning.callbacks import Callback, EarlyStopping
import optuna
import os
import json

# Visualization settings
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)
torch.set_float32_matmul_precision('medium')

In [2]:
target_columns = [
    'Temperature','Precipitation_accumulated','Humidity', 'Wind_Speed_kmh',
    'Soil_Moisture', 'Soil_Temperature', 'Wind_Dir_Sin', 'Wind_Dir_Cos'
]

DATA_FILE_PATH = "../data/ground_station_clean.csv"
df = pd.read_csv(DATA_FILE_PATH)

In [3]:
encoders = {
    "cyclic": {
        "past": ["month", "dayofyear", "day", "hour", "minute"],
        "future": ["month", "dayofyear", "day", "hour", "minute"]
    },
    "transformer": Scaler(),
    "datetime_attribute": {
        "past": ["year"],
        "future": ["year"]
    }
}


In [4]:
class PatchedPruningCallback(PyTorchLightningPruningCallback, Callback):
    pass

# Create directory to save iteration results
results_output_dir = "optuna_iteration_metrics"
os.makedirs(results_output_dir, exist_ok=True)

In [5]:
def objective(trial):
    input_chunk_length = trial.suggest_int("input_chunk_length", 24, 72)
    output_chunk_length = trial.suggest_int("output_chunk_length", 1, 24)
    dropout = trial.suggest_float("dropout", 0.1, 0.5)
    batch_size = trial.suggest_int("batch_size", 64, 256)
    lr = trial.suggest_float("lr", 1e-6, 5e-3, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-2, log=True)

    hidden_size = trial.suggest_categorical('hidden_size', [32, 64, 128])
    ff_size = trial.suggest_int('ff_size', hidden_size, hidden_size * 4)
    num_blocks = trial.suggest_int('num_blocks', 1, 4)
    activation = trial.suggest_categorical('activation', ['ReLU', 'GELU'])
    norm_type = trial.suggest_categorical('norm_type', ['LayerNormNoBias', 'LayerNorm', 'TimeBatchNorm2d'])
    normalize_before = trial.suggest_categorical('normalize_before', [True, False])

    # Try different loss functions
    loss_functions = {
        'mse': torch.nn.MSELoss(),
        'mae': torch.nn.L1Loss(),
        'huber': torch.nn.HuberLoss(delta=1.0),
        'smoothl1': torch.nn.SmoothL1Loss()
    }
    loss_name = trial.suggest_categorical("loss_fn", list(loss_functions.keys()))

    # Try different optimizers
    optimizer_names = ['Adam', 'AdamW', 'RMSprop', 'SGD']
    optimizer_name = trial.suggest_categorical("optimizer", optimizer_names)
    optimizer_dict = {
        'Adam': torch.optim.Adam,
        'AdamW': torch.optim.AdamW,
        'RMSprop': torch.optim.RMSprop,
        'SGD': torch.optim.SGD
    }

    early_stopper = EarlyStopping("val_loss", min_delta=0.0005, patience=15, verbose=True)
    prunner = PatchedPruningCallback(trial, monitor="val_loss")
    pl_trainer_kwargs = {
        "accelerator": "auto",
        "callbacks": [early_stopper, prunner],
    }
    n = int(len(df) * 0.8)
    train_df_fold, val_df_fold = df.iloc[:n], df.iloc[n:]
    train_fold = TimeSeries.from_dataframe(train_df_fold, time_col="Timestamp", value_cols=target_columns, freq='1h')
    val_fold = TimeSeries.from_dataframe(val_df_fold, time_col="Timestamp", value_cols=target_columns, freq='1h')

    print(f"\nStarting Trial {trial.number}")
    print(f"Hyperparameters: {trial.params}")
    print("\nTraining the model...")
    print(f"Train set: {len(train_fold)} samples")
    print(f"Validation set: {len(val_fold)} samples")

    scaler = Scaler()
    scaler = scaler.fit(train_fold)
    train_scaled = scaler.transform(train_fold)
    val_scaled = scaler.transform(val_fold)

    _work_dir = "/home/eduardo/Documentos/Water-Cycle-Neural-Network/darts_logs/"
    _model_name = "model_optuna_temp"
    os.makedirs(_work_dir, exist_ok=True)

    # Additional optimizer kwargs based on optimizer type
    opt_kwargs = {"lr": lr, "weight_decay": weight_decay}
    if optimizer_name == 'SGD':
        opt_kwargs['momentum'] = 0.9

    model = TSMixerModel(
        model_name=_model_name,
        work_dir=_work_dir,
        input_chunk_length=input_chunk_length,
        output_chunk_length=output_chunk_length,
        hidden_size=hidden_size,
        activation=activation,
        num_blocks=num_blocks,
        norm_type=norm_type,
        ff_size=ff_size,
        normalize_before=normalize_before,
        n_epochs=30,
        batch_size=batch_size,
        dropout=dropout,
        add_encoders=encoders,
        pl_trainer_kwargs=pl_trainer_kwargs,
        loss_fn=loss_functions[loss_name],
        optimizer_cls=optimizer_dict[optimizer_name],
        lr_scheduler_cls=torch.optim.lr_scheduler.ReduceLROnPlateau,
        lr_scheduler_kwargs={"mode": "min", "factor": 0.3, "patience": 5, "min_lr": 1e-7},
        save_checkpoints=True,
        show_warnings=True,
        force_reset=True,
        optimizer_kwargs=opt_kwargs,
    )
    model.fit(
        series=train_scaled,
        val_series=val_scaled,
        verbose=False,
        dataloader_kwargs={"num_workers": 9},
    )

    try:
        loaded_model = TSMixerModel.load_from_checkpoint(model_name=_model_name, work_dir=_work_dir, best=True)
        print(f"Model loaded from checkpoint for trial {trial.number}")
    except FileNotFoundError:
        print(f"Checkpoint not found for {_model_name} in {_work_dir}. Using the in-memory trained model.")
        loaded_model = model

    forecasts = loaded_model.historical_forecasts(
        val_scaled,
        forecast_horizon=output_chunk_length,
        stride=1,
        retrain=False,
        verbose=False
    )

    forecasts_t = scaler.inverse_transform(forecasts)
    s = scaler.inverse_transform(val_scaled)
    metrics = {}
    print("Starting time series verification for consistency...")
    try:
        for target in target_columns:
            metrics[target] = {
                'MSE': mse(s[target], forecasts_t[target]),
                'RMSE': rmse(s[target], forecasts_t[target]),
                'MAE': mae(s[target], forecasts_t[target]),
                'R2': r2_score(s[target], forecasts_t[target]),
                'SMAPE': smape(s[target], forecasts_t[target]),
            }
        metrics_df = pd.DataFrame(metrics).T
        print("\nPerformance metrics:")
        print(metrics_df)
    except Exception as e:
        print(e)

    overall_mae_val = mae(val_scaled, forecasts)
    print(f"The MAE for this fold was {overall_mae_val}")

    trial_dict = {
        "trial_number": trial.number,
        "input_chunk_length": input_chunk_length,
        "output_chunk_length": output_chunk_length,
        "hidden_size": hidden_size,
        "num_blocks": num_blocks,
        "ff_size": ff_size,
        "norm_type": norm_type,
        "normalize_before": normalize_before,
        "activation": activation,
        "dropout": dropout,
        "batch_size": batch_size,
        "lr": lr,
        "loss_function": loss_name,
        "optimizer": optimizer_name,
        "overall_mae_val": overall_mae_val,
        "metrics": metrics_df.to_dict(),
    }
    json_path = os.path.join(results_output_dir, f"trial_{trial.number}.json")
    with open(json_path, 'w') as f:
        json.dump(trial_dict, f, indent=4)
    print(f"Results of trial {trial.number} saved in {json_path}")
    return overall_mae_val if not np.isnan(overall_mae_val) else float("inf")


In [6]:
def print_callback(study, trial):
    print(f"Current value: {trial.value}, Current params: {trial.params}")
    print(f"Best value: {study.best_value}, Best params: {study.best_trial.params}")

In [7]:
study = optuna.create_study(direction="minimize")
num_hyperparams = 14
n_trials = 7 * num_hyperparams
study.optimize(objective, n_trials=n_trials, callbacks=[print_callback])

[I 2025-05-27 08:34:49,593] A new study created in memory with name: no-name-b7eb1a4a-1739-4790-9fdf-1888f50f4343



Starting Trial 0
Hyperparameters: {'input_chunk_length': 30, 'output_chunk_length': 9, 'dropout': 0.37326395152584924, 'batch_size': 164, 'lr': 3.0032112604548485e-05, 'weight_decay': 0.0008016232760992192, 'hidden_size': 32, 'ff_size': 39, 'num_blocks': 2, 'activation': 'GELU', 'norm_type': 'LayerNormNoBias', 'normalize_before': True, 'loss_fn': 'mae', 'optimizer': 'SGD'}

Training the model...
Train set: 6647 samples
Validation set: 1662 samples


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Metric val_loss improved. New best score: 0.644
Metric val_loss improved by 0.009 >= min_delta = 0.0005. New best score: 0.635
Metric val_loss improved by 0.009 >= min_delta = 0.0005. New best score: 0.626
Metric val_loss improved by 0.009 >= min_delta = 0.0005. New best score: 0.618
Metric val_loss improved by 0.009 >= min_delta = 0.0005. New best score: 0.609
Metric val_loss improved by 0.008 >= min_delta = 0.0005. New best score: 0.601
Metric val_loss improved by 0.008 >= min_delta = 0.0005. New best score: 0.592
Metric val_loss improved by 0.008 >= min_delta = 0.0005. New best score: 0.584
Metric val_loss improved by 0.008 >= min_delta = 0.0005. New best score: 0.576
Metric val_loss improved by 0.008 >= min_delta = 0.0005. New best score: 0.568
Metric val_loss improved by 0.008 >= min_delta = 0.0005. New best score: 0.561
Metr

Model loaded from checkpoint for trial 0


/home/eduardo/Documentos/Water-Cycle-Neural-Network/venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
[I 2025-05-27 08:35:18,227] Trial 0 finished with value: 0.4158687948094733 and parameters: {'input_chunk_length': 30, 'output_chunk_length': 9, 'dropout': 0.37326395152584924, 'batch_size': 164, 'lr': 3.0032112604548485e-05, 'weight_decay': 0.0008016232760992192, 'hidden_size': 32, 'ff_size': 39, 'num_blocks': 2, 'activation': 'GELU', 'norm_type': 'LayerNormNoBias', 'normalize_before': True, 'loss_fn': 'mae', 'optimizer': 'SGD'}. Best is trial 0 with value: 0.4158687948094733.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

Current value: 0.031796957961971344, Current params: {'input_chunk_length': 27, 'output_chunk_length': 2, 'dropout': 0.474292277444898, 'batch_size': 74, 'lr': 0.002837743466978628, 'weight_decay': 0.0005059426937479169, 'hidden_size': 128, 'ff_size': 186, 'num_blocks': 2, 'activation': 'GELU', 'norm_type': 'TimeBatchNorm2d', 'normalize_before': True, 'loss_fn': 'smoothl1', 'optimizer': 'AdamW'}
Best value: 0.07853027585428113, Best params: {'input_chunk_length': 27, 'output_chunk_length': 1, 'dropout': 0.45087323478159125, 'batch_size': 67, 'lr': 0.004216838508548973, 'weight_decay': 0.000358724351349336, 'hidden_size': 128, 'ff_size': 132, 'num_blocks': 1, 'activation': 'ReLU', 'norm_type': 'TimeBatchNorm2d', 'normalize_before': True, 'loss_fn': 'smoothl1', 'optimizer': 'AdamW'}

Starting Trial 96
Hyperparameters: {'input_chunk_length': 30, 'output_chunk_length': 1, 'dropout': 0.4995583955569816, 'batch_size': 83, 'lr': 0.0025941638835990837, 'weight_decay': 0.0003504290638404204, 'h

[I 2025-05-27 09:40:29,452] Trial 96 pruned. Trial was pruned at epoch 0.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Current value: 0.037128406052418135, Current params: {'input_chunk_length': 30, 'output_chunk_length': 1, 'dropout': 0.4995583955569816, 'batch_size': 83, 'lr': 0.0025941638835990837, 'weight_decay': 0.0003504290638404204, 'hidden_size': 128, 'ff_size': 205, 'num_blocks': 3, 'activation': 'GELU', 'norm_type': 'TimeBatchNorm2d', 'normalize_before': True, 'loss_fn': 'smoothl1', 'optimizer': 'AdamW'}
Best value: 0.07853027585428113, Best params: {'input_chunk_length': 27, 'output_chunk_length': 1, 'dropout': 0.45087323478159125, 'batch_size': 67, 'lr': 0.004216838508548973, 'weight_decay': 0.000358724351349336, 'hidden_size': 128, 'ff_size': 132, 'num_blocks': 1, 'activation': 'ReLU', 'norm_type': 'TimeBatchNorm2d', 'normalize_before': True, 'loss_fn': 'smoothl1', 'optimizer': 'AdamW'}

Starting Trial 97
Hyperparameters: {'input_chunk_length': 32, 'output_chunk_length': 2, 'dropout': 0.4784606331079146, 'batch_size': 82, 'lr': 0.003285156538232187, 'weight_decay': 0.0018932135041158096, '

Metric val_loss improved. New best score: 0.033
[I 2025-05-27 09:40:32,229] Trial 97 pruned. Trial was pruned at epoch 1.


Current value: 0.025987424300693285, Current params: {'input_chunk_length': 32, 'output_chunk_length': 2, 'dropout': 0.4784606331079146, 'batch_size': 82, 'lr': 0.003285156538232187, 'weight_decay': 0.0018932135041158096, 'hidden_size': 128, 'ff_size': 165, 'num_blocks': 2, 'activation': 'GELU', 'norm_type': 'TimeBatchNorm2d', 'normalize_before': True, 'loss_fn': 'smoothl1', 'optimizer': 'AdamW'}
Best value: 0.07853027585428113, Best params: {'input_chunk_length': 27, 'output_chunk_length': 1, 'dropout': 0.45087323478159125, 'batch_size': 67, 'lr': 0.004216838508548973, 'weight_decay': 0.000358724351349336, 'hidden_size': 128, 'ff_size': 132, 'num_blocks': 1, 'activation': 'ReLU', 'norm_type': 'TimeBatchNorm2d', 'normalize_before': True, 'loss_fn': 'smoothl1', 'optimizer': 'AdamW'}


In [8]:
print(f"Best Value (Minimum): {study.best_value}")
print(f"Best Parameters: {study.best_params}")

Best Value (Minimum): 0.07853027585428113
Best Parameters: {'input_chunk_length': 27, 'output_chunk_length': 1, 'dropout': 0.45087323478159125, 'batch_size': 67, 'lr': 0.004216838508548973, 'weight_decay': 0.000358724351349336, 'hidden_size': 128, 'ff_size': 132, 'num_blocks': 1, 'activation': 'ReLU', 'norm_type': 'TimeBatchNorm2d', 'normalize_before': True, 'loss_fn': 'smoothl1', 'optimizer': 'AdamW'}


In [9]:
best_dict = {
    "best_value": study.best_value,
    "best_params": study.best_params,
}
json_path = os.path.join(results_output_dir, "best_trial.json")
with open(json_path, 'w') as f:
    json.dump(best_dict, f, indent=4)
print(f"Best results saved in {json_path}")


Best results saved in optuna_iteration_metrics/best_trial.json
